In [126]:
import numpy as np
from numpy.linalg import norm, cholesky, solve, inv
from numpy import matmul, eye


In [127]:
def generate_householder_mat(dim):
    rng = np.random.default_rng(52)

    w = rng.random(dim)
    w = w / norm(w, ord=None) 

    lambdas = rng.random(dim)
    lambda_mat = eye(dim)

    for i in range(dim):
        lambda_mat[i, i] = lambdas[i]
    print(lambdas, "\n", lambda_mat)

    H = eye(dim) - 2 * matmul(w.reshape(dim, -1), w.reshape(-1, dim))
    # print(w.reshape(dim, -1), w.reshape(-1, dim))
    print(H)

    return matmul(matmul(H, lambda_mat), H.T)


size_H = 10
test_hh = generate_householder_mat(size_H)
print(test_hh)


[0.89601131 0.47378689 0.11715695 0.07880772 0.98278754 0.06445595
 0.48948325 0.40900674 0.76510318 0.07251445] 
 [[0.89601131 0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.47378689 0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.11715695 0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.07880772 0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.98278754 0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.06445595
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.48948325 0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.40900674 0.         0.        ]
 [0.         0.      

In [128]:
# def norm(vec):
#   sum = 0
#   for i in vec:
#     sum += i*i 
#   return sum**(1/2)

In [129]:
def generate_positive_definite_matrix(size: int) -> np.ndarray:
    """
    Generate random symmetric positive definite matrix:
    1. Create random matrix A
    2. Compute A^T * A (always positive semi-definite)
    3. Add diagonal dominance to ensure positive definite
    """
    rng = np.random.default_rng(52)
    
    # Create random matrix
    A = rng.random((size, size))
    
    # Make symmetric positive definite
    matrix = A @ A.T  # Ensures positive semi-definite
    
    # Add to diagonal to ensure positive definite
    matrix += size//2 * eye(size)
    
    return matrix

In [130]:
# mat_size = 3
# mat = generate_positive_definite_matrix(3)

# print(mat)
# mat = np.ndarray((2, 2), dtype=np.int64)


def inverse_iteration(A: np.ndarray, tolerance: np.float64 = 1e-13, max_iter: int = 500):
    dim = A.shape[0]

    # initial guess
    x_prev = np.random.rand(dim)
    # print(x_prev)

    sigma = 0
    sigma_prev = 0

    # decompose A into L*U using cholesky
    L = cholesky(A, upper=False)
    U = L.T

    A_inv = np.linalg.inv(A)

    # U = np.linalg.cholesky(A, upper=True)

    # print(L @ L.T) # A matr

    for _ in range(max_iter):
        x_norm = norm(x_prev, ord=None)
        # print(norm)
        nu = x_prev / x_norm  # +- eigenvec

        # L*y = nu
        # U*x = y
        y = solve(L, nu)
        x_next = solve(U, y)
        # x_next = matmul(A_inv, nu)

        sigma_prev = sigma
        sigma = matmul(nu, x_next)  # eigenvalue

        if abs(sigma - sigma_prev) < tolerance and abs(sigma - sigma_prev) != 0:
            # print(_)
            return nu, 1 / sigma

        sigma_prev = sigma
        x_prev = x_next

    return nu, 1 / sigma


eigvec_1, eigval_1 = inverse_iteration(test_hh)

print(eigval_1, eigvec_1)


0.06445594702461908 [ 0.35776894  0.33363542  0.30186382  0.00371243  0.31972941 -0.4373786
  0.23127958  0.00920181  0.14162437  0.54990356]


In [131]:
def inverse_iteration_with_shift_one(
    A: np.ndarray, tolerance: np.float64 = 1e-13, max_iter: int = 500
):
    dim = A.shape[0]

    # initial guess
    x_prev = np.random.rand(dim)
    # print(x_prev)

    sigma = 0
    sigma_prev = 0

    # decompose A into L*U using cholesky
    L = cholesky(A, upper=False)
    U = L.T

    g_1, _ = inverse_iteration(A)
    # print(g_1, _)

    # U = np.linalg.cholesky(A, upper=True)

    # print(np.matmul(g_1.reshape(dim, -1), g_1.reshape(-1, dim)))
    eyeminus = eye(dim) - matmul(g_1.reshape(dim, -1), g_1.reshape(-1, dim))

    # print(L @ L.T) # A matr

    for i in range(max_iter):
        x_norm = norm(x_prev, ord=None)
        # print(norm)
        nu = x_prev / x_norm  # +- eigenvec

        # f = E - g_1*g_1^T
        # L*y = f * nu
        # U*x = y

        y = solve(
            L,
            matmul(eyeminus, nu),
        )
        x_next = solve(U, y)

        # print(g_1.reshape(dim, -1).shape, g_1.reshape(-1, dim).shape)
        # x_next = matmul(
        #     matmul(
        #         np.linalg.inv(A),
        #         np.eye(dim) - np.matmul(g_1.reshape(dim, -1), g_1.reshape(-1, dim)),
        #     ),
        #     nu,
        # )

        sigma_prev = sigma
        sigma = matmul(nu, x_next)  # eigenvalue

        if abs(sigma - sigma_prev) < tolerance:
          # print(i)
          return nu, 1 / sigma, i

        sigma_prev = sigma
        x_prev = x_next

    return nu, 1 / sigma, max_iter


In [132]:
print(inverse_iteration_with_shift_one(test_hh))

(array([ 0.34968166,  0.32609367,  0.29504026,  0.00362918,  0.312502  ,
        0.54990399,  0.22605156,  0.0089938 ,  0.13842299, -0.46252643]), np.float64(0.07251445258539525), 164)


In [133]:
print(np.linalg.eig(test_hh))

EigResult(eigenvalues=array([0.98278754, 0.89601131, 0.76510318, 0.48948325, 0.47378689,
       0.40900674, 0.11715695, 0.06445595, 0.07251445, 0.07880772]), eigenvectors=array([[ 2.03314702e-01, -7.72496107e-01,  9.00583941e-02,
        -1.47069793e-01, -2.12157482e-01, -5.85139311e-03,
         1.91954040e-01,  3.57768783e-01, -3.49681846e-01,
         2.36072267e-03],
       [ 1.89599987e-01,  2.12157482e-01,  8.39834511e-02,
        -1.37149112e-01,  8.02153729e-01, -5.45668388e-03,
         1.79005665e-01,  3.33635275e-01, -3.26093848e-01,
         2.20147871e-03],
       [ 1.71544662e-01,  1.91954040e-01,  7.59858319e-02,
        -1.24088606e-01, -1.79005665e-01, -4.93705199e-03,
        -8.38040778e-01,  3.01863684e-01, -2.95040416e-01,
         1.99183516e-03],
       [ 2.10972050e-03,  2.36072267e-03,  9.34502217e-04,
        -1.52608815e-03, -2.20147871e-03, -6.07177142e-05,
         1.99183516e-03,  3.71243263e-03, -3.62851752e-03,
        -9.99975504e-01],
       [-8.183025

## Testing

In [134]:
def unit_vector(vector):
    """ Returns the unit vector of the vector.  """
    return vector / norm(vector)

def angle_between(v1, v2):
    """ Returns the angle in radians between vectors 'v1' and 'v2'::

            >>> angle_between((1, 0, 0), (0, 1, 0))
            1.5707963267948966
            >>> angle_between((1, 0, 0), (1, 0, 0))
            0.0
            >>> angle_between((1, 0, 0), (-1, 0, 0))
            3.141592653589793
    """
    v1_u = unit_vector(v1)
    v2_u = unit_vector(v2)
    return np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0))

In [137]:
def TEST_generate_householder_mat(dim, lambda_range):
    rng = np.random.default_rng()

    w = rng.uniform(-lambda_range, lambda_range, dim)
    w = rng.random(dim)
    w = w / norm(w, ord=None)

    lambdas = rng.uniform(-lambda_range, lambda_range, dim)
    lambdas = rng.random(dim)
    lambda_mat = eye(dim)

    for i in range(dim):
        lambda_mat[i, i] = lambdas[i]
    # print(lambdas, "\n", lambda_mat)

    idx_second_min = np.where(lambdas == np.sort(lambdas)[1])
    # print(idx_second_min[0])
    # second min
    second_lam = np.sort(lambdas)[1]
    # print(second_lam)
    H = np.eye(dim) - 2 * matmul(w.reshape(dim, -1), w.reshape(-1, dim))
    second_g = H[:, 1]
    # print(second_g)
    # print(lambdas)
    # print(H)

    return (
        matmul(matmul(H, lambda_mat), H.T),
        lambdas[idx_second_min[0]],
        H[:, idx_second_min[0]],
    )


def calculate_accuracy_measure(
    A: np.ndarray, x: np.ndarray, lambda_val: float
) -> float:
    """
    Calculate accuracy measure r = ||Ax - λx||_∞

    Args:
        A: Input matrix
        x: Eigenvector
        lambda_val: Eigenvalue
    Returns:
        r: Accuracy measure (max absolute residual)
    """
    # Calculate residual vector r = Ax - λx
    Ax = matmul(A, x)
    lambda_x = lambda_val * x
    residual = Ax - lambda_x

    # Find max absolute value
    r = np.max(np.abs(residual))

    return r


def TEST(size_range, lambda_range, lambda_diff=10 ** (-5)):
    rng = np.random.default_rng()
    cnt = 0

    lambdas_diffs = []
    vectors_diffs = []
    iterations = []

    while cnt < 10:
        size = rng.integers(2, size_range+1)

        try:
            A, eigenval, eigenvec = TEST_generate_householder_mat(size, lambda_range)

            vec, val, iter = inverse_iteration_with_shift_one(A)

            # print(vec, val, "\n", iter)
            # print(eigenvec, eigenval)

            cnt += 1
            # print(vec, eigenvec.reshape(-1))
            vectors_diffs.append(
                min(
                    abs(angle_between(vec, eigenvec.reshape(-1))),
                    abs(angle_between(-vec, eigenvec.reshape(-1))),
                ),
            )
            lambdas_diffs.append(abs(val - eigenval))
            iterations.append(iter)
        except np.linalg.LinAlgError:
            print("except")

    print(
        size_range,
        " -- ",
        lambda_range,
        " -- ",
        f"eps={10**(-13)}",
        " -- ",
        np.mean(lambdas_diffs),
        " -- ",
        np.mean(vectors_diffs),
        " -- ",
        calculate_accuracy_measure(A, vec, val),
        " -- ",
        np.mean(iterations),
    )

for size_range in [10, 30, 50]:
  print(size_range)
  for eigenvals in [2, 50]:
    print(eigenvals)
    for eig_diffs in [10**(-5), 10**(-8)]:
      TEST(size_range, eigenvals, eig_diffs)
  
# TEST(10, 2)


10
2
10  --  2  --  eps=1e-13  --  3.0425093228148015e-07  --  0.0035863072076428885  --  6.28754635301973e-05  --  95.5
10  --  2  --  eps=1e-13  --  3.4536325455164494e-07  --  0.0070844014966237505  --  1.4782582623273921e-08  --  88.4
50
10  --  50  --  eps=1e-13  --  4.51090553799105e-14  --  7.930001650839452e-07  --  7.143833413020273e-08  --  82.5
10  --  50  --  eps=1e-13  --  2.855771175092059e-14  --  6.296090216987544e-07  --  7.878551082363128e-09  --  40.3
30
2
30  --  2  --  eps=1e-13  --  5.933101232535875e-15  --  3.7025702426256977e-07  --  1.9689704533476515e-08  --  49.7
30  --  2  --  eps=1e-13  --  1.372721381009967e-14  --  5.310334714677528e-07  --  9.569884426574582e-09  --  67.0
50
30  --  50  --  eps=1e-13  --  1.1481155089143602e-11  --  2.0218931034203043e-05  --  5.752061697176375e-09  --  118.1
30  --  50  --  eps=1e-13  --  1.9184357846827037e-07  --  0.0025866679660719446  --  1.890276459665108e-09  --  120.5
50
2
50  --  2  --  eps=1e-13  --  5.7958800